# Compute Rule Counts

In [ ]:
!python count_cfg_freq.py parse_train.dat > cfg.counts

In [14]:
import pandas as pd

df = pd.read_csv('cfg.counts', sep=' ', names=['Count', 'Rule Type', 'Rule', 'Token'])

# Find Rare Words

In [50]:
unaries = df[df['Rule Type'] == 'UNARYRULE']

In [61]:
from collections import defaultdict

counts = defaultdict(int)

for count, token in unaries[['Count', 'Token']].values:
    counts[token] += int(count)

In [67]:
rares = set(token for token, count in counts.items() if count < 5)

# Replace All Occurrences of Rare Words in Original Training Set with _RARE_

In [88]:
with open('parse_train.dat', 'r') as f:
    lines = f.readlines()
    
import json
trees = [json.loads(line) for line in lines]

In [71]:
def rarify(tree, rares):
    """Produce a new tree with rare leaves replaced with the symbol _RARE_"""
    def rarify_iter(tree):
        # Leaf?
        if isinstance(tree, basestring):
            return [tree] if tree not in rares else ['_RARE_']

        # Binary rule?
        elif len(tree) == 3:
            return tree[:1] + [rarify_iter(tree[1])] + [rarify_iter(tree[2])]

        # Unary rule?
        elif len(tree) == 2:
            return tree[:1] + rarify_iter(tree[1])
    
    return rarify_iter(tree)

# Write Training Data Back with _RARE_

In [110]:
with open('parse_train+rare.dat', 'w') as f:
    for tree in trees:
        json.dump(rarify(tree, rares), f)
        f.write('\n')

# Rerun Counts

In [112]:
!python count_cfg_freq.py parse_train+rare.dat > cfg.counts